# Imports

In [1]:
%run setup.py

In [3]:
import pandas as pd
import numpy as np
import joblib
from pathlib import Path
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
import lightgbm as lgb

from sklearn.metrics import average_precision_score, roc_auc_score, f1_score, precision_recall_curve, auc

from typing import List, Tuple
import optuna

from thesis_code.data_analysis.ml_pipeline import (
    load_parquet,
    load_multiple_parquets,
    protein_level_split,
    preprocess,
    calculate_class_weights,
    train_model,
    evaluate_model,
    save_model,
    load_model
    )

RANDOM_STATE = 42

# Load Data

In [4]:
df_full = load_parquet("/home/user_stel/project_thesis/data/derived/df_feat_context_v1.parquet")
df_surface = load_parquet("/home/user_stel/project_thesis/data/derived/df_ml_context_v1.parquet")

print(df_full.shape)
print(df_surface.shape)


(183890, 43)
(106100, 43)


## Sanity Checks

In [5]:
def check_dataset(
    df: pd.DataFrame,
    target_col: str = "IBS",
    required_cols: List[str] | None = None
):
    """
    Basic sanity checks for ML datasets.

    Parameters
    ----------
    df : pd.DataFrame
    target_col : str
    required_cols : list, optional
    """

    if target_col not in df.columns:
        raise ValueError(f"Target column '{target_col}' not found")

    if required_cols is not None:
        missing = set(required_cols) - set(df.columns)
        if missing:
            raise ValueError(f"Missing required columns: {missing}")

    if df[target_col].isna().any():
        raise ValueError("Target column contains NaNs")

    print("Dataset check passed:")
    print(f"  Samples: {len(df):,}")
    print(f"  IBS positives: {df[target_col].sum():,}")
    print(f"  IBS ratio: {df[target_col].mean():.4f}")


In [6]:
print("Full-surface IBS ratio:", df_full["IBS"].mean())
print("Surface-only IBS ratio:", df_surface["IBS"].mean())

df_full["IBS"].value_counts()

Full-surface IBS ratio: 0.1516286910653108
Surface-only IBS ratio: 0.20555136663524975


IBS
False    156007
True      27883
Name: count, dtype: int64

# Split train/test/val 

In [7]:
# Step 1: inspect
for i, col in enumerate(df_full.columns):
    print(f"{i:2d}  {col}")

# Step 2: decide
DROP_COLS = [
    "IBS",
    "domain",
    "cathpdb",
    "pdb",
    "uniprot_acc",
    "uniprot_id",
    "domain",
    "residue_name",
    "chain_id",
    "residue_number",
    "Experimental Method",
    "origin",
    "location",
    "taxon",
    "data_type",
    "resolution",
    "S35", "S60", "S95", "S100",
    "uniref50", "uniref90", "uniref100",
    "neighboursList",
    "neighbors"
]

 0  domain
 1  cathpdb
 2  pdb
 3  uniprot_acc
 4  uniprot_id
 5  residue_name
 6  IBS
 7  chain_id
 8  residue_number
 9  b_factor
10  sec_struc
11  sec_struc_full
12  prot_block
13  data_type
14  Experimental Method
15  resolution
16  RSA_total_freesasa_tien
17  convhull_vertex
18  protrusion
19  is_hydrophobic_protrusion
20  is_co_insertable
21  neighboursList
22  density
23  exposed
24  S35
25  S60
26  S95
27  S100
28  uniref50
29  uniref90
30  uniref100
31  origin
32  location
33  taxon
34  neighbors
35  n_neighbors
36  neighbor_frac_exposed
37  neighbor_frac_hydrophobic
38  neighbor_frac_charged
39  neighbor_frac_polar
40  neighbor_frac_aromatic
41  neighbor_frac_small
42  neighbor_mean_RSA


In [8]:
df_full = df_full.reset_index(drop=True)
df_surface = df_surface.reset_index(drop=True)

In [9]:
train_prot, val_prot, test_prot = protein_level_split(df_full, "cathpdb")

df_train = df_full[df_full["cathpdb"].isin(train_prot)]
df_val   = df_full[df_full["cathpdb"].isin(val_prot)]
df_test  = df_full[df_full["cathpdb"].isin(test_prot)]

# Preprocessing

In [ ]:
X_train, y_train = preprocess(
    df_train,
    target_col="IBS",
    drop_cols=DROP_COLS
)

X_val, y_val = preprocess(
    df_val,
    target_col="IBS",
    drop_cols=DROP_COLS
)

X_test, y_test = preprocess(
    df_test,
    target_col="IBS",
    drop_cols=DROP_COLS
)

# Hyperparameter Tuning

In [11]:
categorical_cols = X_train.select_dtypes(include="object").columns.tolist()
numeric_cols = X_train.select_dtypes(include=np.number).columns.tolist()

In [ ]:
print(X_train.dtypes)

b_factor                     float64
sec_struc                     object
sec_struc_full                object
prot_block                    object
RSA_total_freesasa_tien      float64
convhull_vertex                 bool
protrusion                      bool
is_hydrophobic_protrusion       bool
is_co_insertable                bool
density                        int64
exposed                         bool
n_neighbors                  float64
neighbor_frac_exposed        float64
neighbor_frac_hydrophobic    float64
neighbor_frac_charged        float64
neighbor_frac_polar          float64
neighbor_frac_aromatic       float64
neighbor_frac_small          float64
neighbor_mean_RSA            float64
dtype: object


### Random Forest

In [13]:
preprocessor = ColumnTransformer(
    transformers=[
        ("num", "passthrough", numeric_cols),
        ("cat", OneHotEncoder(
            handle_unknown="ignore",
            sparse_output=False
        ), categorical_cols)
    ]
)

rf_pipeline = Pipeline(
    steps=[
        ("preprocess", preprocessor),
        ("model", RandomForestClassifier(
            random_state=RANDOM_STATE,
            n_jobs=-1
        ))
    ]
)

In [14]:
def rf_objective(trial):

    params = {
        "model__n_estimators": trial.suggest_int("model__n_estimators", 300, 800),
        "model__max_depth": trial.suggest_int("model__max_depth", 5, 18),
        "model__min_samples_split": trial.suggest_int("model__min_samples_split", 5, 20),
        "model__min_samples_leaf": trial.suggest_int("model__min_samples_leaf", 5, 20),
        "model__max_features": trial.suggest_categorical(
            "model__max_features", ["sqrt", "log2"]
        )
    }

    model = rf_pipeline.set_params(**params)

    # Tune positive-class importance
    pos_scale = trial.suggest_float("pos_scale", 0.5, 2.0)
    class_weights = calculate_class_weights(y_train, pos_scale=pos_scale)
    model.set_params(model__class_weight=class_weights)

    model.fit(X_train, y_train)

    y_val_proba = model.predict_proba(X_val)[:, 1]
    return average_precision_score(y_val, y_val_proba)

In [16]:
study_rf = optuna.create_study(direction="maximize")
study_rf.optimize(rf_objective, n_trials=25)

[I 2026-02-10 20:38:24,890] A new study created in memory with name: no-name-12c4bf12-d325-4995-b4a2-1fc6dbb1f7af
[I 2026-02-10 20:39:02,200] Trial 0 finished with value: 0.3416985751541191 and parameters: {'model__n_estimators': 619, 'model__max_depth': 9, 'model__min_samples_split': 12, 'model__min_samples_leaf': 8, 'model__max_features': 'sqrt', 'pos_scale': 1.6544023636914618}. Best is trial 0 with value: 0.3416985751541191.
[I 2026-02-10 20:39:41,755] Trial 1 finished with value: 0.3763907837763151 and parameters: {'model__n_estimators': 624, 'model__max_depth': 15, 'model__min_samples_split': 6, 'model__min_samples_leaf': 9, 'model__max_features': 'log2', 'pos_scale': 1.9654197554955117}. Best is trial 1 with value: 0.3763907837763151.
[I 2026-02-10 20:40:07,671] Trial 2 finished with value: 0.29404364528311017 and parameters: {'model__n_estimators': 729, 'model__max_depth': 5, 'model__min_samples_split': 15, 'model__min_samples_leaf': 19, 'model__max_features': 'sqrt', 'pos_scal

In [17]:
import importlib
import thesis_code.data_analysis.ml_pipeline as ml_pipeline
importlib.reload(ml_pipeline)


<module 'thesis_code.data_analysis.ml_pipeline' from '/home/user_stel/project_thesis/thesis_code/data_analysis/ml_pipeline.py'>

In [26]:
best_params = study_rf.best_params.copy()
print("Best params:", best_params)

pos_scale = best_params.pop("pos_scale", None)

if pos_scale is not None:
    class_weights = {0: 1.0, 1: pos_scale}
else:
    class_weights = "balanced"


best_rf = rf_pipeline.set_params(
    **best_params,
    model__class_weight=class_weights
)


Best params: {'model__n_estimators': 508, 'model__max_depth': 17, 'model__min_samples_split': 19, 'model__min_samples_leaf': 5, 'model__max_features': 'log2', 'pos_scale': 0.9787537479949556}


In [27]:
ml_pipeline.save_model(best_rf, "rf_model")

Model saved to: /home/user_stel/project_thesis/thesis_code/data_analysis/models/rf_model.joblib


PosixPath('/home/user_stel/project_thesis/thesis_code/data_analysis/models/rf_model.joblib')

In [ ]:
best_rf.get_params().keys() #sanity check

dict_keys(['memory', 'steps', 'transform_input', 'verbose', 'preprocess', 'model', 'preprocess__force_int_remainder_cols', 'preprocess__n_jobs', 'preprocess__remainder', 'preprocess__sparse_threshold', 'preprocess__transformer_weights', 'preprocess__transformers', 'preprocess__verbose', 'preprocess__verbose_feature_names_out', 'preprocess__num', 'preprocess__cat', 'preprocess__cat__categories', 'preprocess__cat__drop', 'preprocess__cat__dtype', 'preprocess__cat__feature_name_combiner', 'preprocess__cat__handle_unknown', 'preprocess__cat__max_categories', 'preprocess__cat__min_frequency', 'preprocess__cat__sparse_output', 'model__bootstrap', 'model__ccp_alpha', 'model__class_weight', 'model__criterion', 'model__max_depth', 'model__max_features', 'model__max_leaf_nodes', 'model__max_samples', 'model__min_impurity_decrease', 'model__min_samples_leaf', 'model__min_samples_split', 'model__min_weight_fraction_leaf', 'model__monotonic_cst', 'model__n_estimators', 'model__n_jobs', 'model__oob_

### Logistic Regression

In [30]:
lr_pipeline = Pipeline(
    steps=[
        ("preprocess", preprocessor),
        ("model", LogisticRegression(
            max_iter=1000,
            random_state=RANDOM_STATE
        ))
    ]
)

param_grid_lr = {
    "model__C": [0.01, 0.1, 1, 10],
    "model__penalty": ["l2"],
    "model__solver": ["lbfgs"]
}

class_weights_lr = calculate_class_weights(y_train)
lr_pipeline.set_params(model__class_weight=class_weights_lr)

Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('num', 'passthrough',
                                                  ['b_factor',
                                                   'RSA_total_freesasa_tien',
                                                   'density', 'n_neighbors',
                                                   'neighbor_frac_exposed',
                                                   'neighbor_frac_hydrophobic',
                                                   'neighbor_frac_charged',
                                                   'neighbor_frac_polar',
                                                   'neighbor_frac_aromatic',
                                                   'neighbor_frac_small',
                                                   'neighbor_mean_RSA']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  ['sec_struc',
                                                   'sec_struc_full',
                                                   'prot_block'])])),
                ('model',
                 LogisticRegression(class_weight={np.int64(0): np.float64(0.5895831603037938),
                                                  np.int64(1): np.float64(3.290703064640742)},
                                    max_iter=1000, random_state=42))])

In [31]:
grid_lr = GridSearchCV(
    estimator=lr_pipeline,     
    param_grid=param_grid_lr,
    scoring="average_precision",
    cv=3,
    n_jobs=-1,
    error_score="raise"        
)

grid_lr.fit(X_train, y_train)


GridSearchCV(cv=3, error_score='raise',
             estimator=Pipeline(steps=[('preprocess',
                                        ColumnTransformer(transformers=[('num',
                                                                         'passthrough',
                                                                         ['b_factor',
                                                                          'RSA_total_freesasa_tien',
                                                                          'density',
                                                                          'n_neighbors',
                                                                          'neighbor_frac_exposed',
                                                                          'neighbor_frac_hydrophobic',
                                                                          'neighbor_frac_charged',
                                                                          'neighbor_frac_polar',
                                                                          'neighbor_frac_aromatic',
                                                                          'neighbor_frac_small',
                                                                          'neighbor_me...
                                                                                       sparse_output=False),
                                                                         ['sec_struc',
                                                                          'sec_struc_full',
                                                                          'prot_block'])])),
                                       ('model',
                                        LogisticRegression(class_weight={np.int64(0): np.float64(0.5895831603037938),
                                                                         np.int64(1): np.float64(3.290703064640742)},
                                                           max_iter=1000,
                                                           random_state=42))]),
             n_jobs=-1,
             param_grid={'model__C': [0.01, 0.1, 1, 10],
                         'model__penalty': ['l2'], 'model__solver': ['lbfgs']},
             scoring='average_precision')

In [32]:
best_lr = grid_lr.best_estimator_
print(grid_lr.best_params_)

{'model__C': 0.01, 'model__penalty': 'l2', 'model__solver': 'lbfgs'}


In [48]:
ml_pipeline.save_model(best_lr, "lr_model")

Model saved to: /home/user_stel/project_thesis/thesis_code/data_analysis/models/lr_model.joblib


PosixPath('/home/user_stel/project_thesis/thesis_code/data_analysis/models/lr_model.joblib')

### Support Vector Machines

In [57]:
preprocessor_svm = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore", sparse_output=False), categorical_cols),
        ("num", StandardScaler(), numeric_cols),
    ],
    remainder="drop"
)

In [58]:
svm_pipeline = Pipeline([
    ("preprocess", preprocessor_svm),  # with StandardScaler
    ("model", SVC(class_weight="balanced", max_iter=5000))])

In [59]:
param_grid_svm = {
    "model__C": [0.1, 1, 10]
    # "model__gamma": ["scale", "auto"]
}

class_weights_svm = calculate_class_weights(y_train)

svm_pipeline.set_params(
    model__class_weight=class_weights_svm
)

Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('cat',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  ['sec_struc',
                                                   'sec_struc_full',
                                                   'prot_block']),
                                                 ('num', StandardScaler(),
                                                  ['b_factor',
                                                   'RSA_total_freesasa_tien',
                                                   'density', 'n_neighbors',
                                                   'neighbor_frac_exposed',
                                                   'neighbor_frac_hydrophobic',
                                                   'neighbor_frac_charged',
                                                   'neighbor_frac_polar',
                                                   'neighbor_frac_aromatic',
                                                   'neighbor_frac_small',
                                                   'neighbor_mean_RSA'])])),
                ('model',
                 SVC(class_weight={np.int64(0): np.float64(0.5895831603037938),
                                   np.int64(1): np.float64(3.290703064640742)},
                     max_iter=5000))])

In [60]:
grid_svm = GridSearchCV(
    estimator=svm_pipeline,        
    param_grid=param_grid_svm,
    scoring="average_precision",
    cv=3,
    n_jobs=-1,
    error_score="raise"
)

grid_svm.fit(X_train, y_train)

/home/user_stel/miniconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:305: ConvergenceWarning: Solver terminated early (max_iter=5000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/home/user_stel/miniconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:305: ConvergenceWarning: Solver terminated early (max_iter=5000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/home/user_stel/miniconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:305: ConvergenceWarning: Solver terminated early (max_iter=5000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/home/user_stel/miniconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:305: ConvergenceWarning: Solver terminated early (max_iter=5000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/home/user_stel/miniconda3/lib/python3.12/site-packages/sklearn/svm/_bas

GridSearchCV(cv=3, error_score='raise',
             estimator=Pipeline(steps=[('preprocess',
                                        ColumnTransformer(transformers=[('cat',
                                                                         OneHotEncoder(handle_unknown='ignore',
                                                                                       sparse_output=False),
                                                                         ['sec_struc',
                                                                          'sec_struc_full',
                                                                          'prot_block']),
                                                                        ('num',
                                                                         StandardScaler(),
                                                                         ['b_factor',
                                                                          'RSA_total_freesasa_tien',
                                                                          'density',
                                                                          'n_neighbors',
                                                                          'neighbor_frac_exposed',
                                                                          'neighbor_frac_hydrophobic',
                                                                          'neighbor_frac_charged',
                                                                          'neighbor_frac_polar',
                                                                          'neighbor_frac_aromatic',
                                                                          'neighbor_frac_small',
                                                                          'neighbor_mean_RSA'])])),
                                       ('model',
                                        SVC(class_weight={np.int64(0): np.float64(0.5895831603037938),
                                                          np.int64(1): np.float64(3.290703064640742)},
                                            max_iter=5000))]),
             n_jobs=-1, param_grid={'model__C': [0.1, 1, 10]},
             scoring='average_precision')

In [61]:
best_svm = grid_svm.best_estimator_
print(grid_svm.best_params_)

{'model__C': 0.1}


In [62]:
ml_pipeline.save_model(best_svm, "svm_model")

Model saved to: /home/user_stel/project_thesis/thesis_code/data_analysis/models/svm_model.joblib


PosixPath('/home/user_stel/project_thesis/thesis_code/data_analysis/models/svm_model.joblib')

### XGBoost

In [66]:
xgb_pipeline = Pipeline(
    steps=[
        ("preprocess", preprocessor),
        ("model", XGBClassifier(
            objective="binary:logistic",
            eval_metric="logloss",
            use_label_encoder=False,
            random_state=RANDOM_STATE,
            n_jobs=-1
        ))
    ]
)

In [70]:
def xgb_objective(trial):

    params = {
        "model__n_estimators": trial.suggest_int("model__n_estimators", 200, 800),
        "model__max_depth": trial.suggest_int("model__max_depth", 3, 10),
        "model__learning_rate": trial.suggest_float(
            "model__learning_rate", 0.01, 0.2, log=True
        ),
        "model__subsample": trial.suggest_float("model__subsample", 0.6, 1.0),
        "model__colsample_bytree": trial.suggest_float(
            "model__colsample_bytree", 0.6, 1.0
        ),
        "model__min_child_weight": trial.suggest_int(
            "model__min_child_weight", 1, 10
        ),
        "model__reg_lambda": trial.suggest_float(
            "model__reg_lambda", 1e-3, 10.0, log=True
        )
    }

    model = xgb_pipeline.set_params(**params)

    class_weights = calculate_class_weights(y_train)
    scale_pos_weight = class_weights[1] / class_weights[0]

    model.set_params(model__scale_pos_weight=scale_pos_weight)

    model.fit(X_train, y_train)

    y_val_proba = model.predict_proba(X_val)[:, 1]
    return average_precision_score(y_val, y_val_proba)


In [71]:
study_xgb = optuna.create_study(direction="maximize")
study_xgb.optimize(xgb_objective, n_trials=50)

[I 2026-01-18 20:00:28,653] A new study created in memory with name: no-name-610eb82a-544b-4caf-8bc5-6063a44af79e
/home/user_stel/miniconda3/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [20:00:29] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
[I 2026-01-18 20:00:44,644] Trial 0 finished with value: 0.3856020086821153 and parameters: {'model__n_estimators': 251, 'model__max_depth': 10, 'model__learning_rate': 0.051889669620576284, 'model__subsample': 0.9112738881725808, 'model__colsample_bytree': 0.9080899900103306, 'model__min_child_weight': 6, 'model__reg_lambda': 0.017218037079039197}. Best is trial 0 with value: 0.3856020086821153.
/home/user_stel/miniconda3/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [20:00:45] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
[I 2

In [72]:
best_xgb = xgb_pipeline.set_params(**study_xgb.best_params)
print(study_xgb.best_params)

{'model__n_estimators': 759, 'model__max_depth': 10, 'model__learning_rate': 0.01323084356551085, 'model__subsample': 0.8971857262596246, 'model__colsample_bytree': 0.8626199585459593, 'model__min_child_weight': 7, 'model__reg_lambda': 0.30385885542295415}


In [73]:
ml_pipeline.save_model(best_xgb, "xgb_model")

Model saved to: /home/user_stel/project_thesis/thesis_code/data_analysis/models/xgb_model.joblib


/home/user_stel/miniconda3/lib/python3.12/pickle.py:572: UserWarning: [20:09:50] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  rv = reduce(self.proto)


PosixPath('/home/user_stel/project_thesis/thesis_code/data_analysis/models/xgb_model.joblib')

### LightGBM

In [76]:
lgb_pipeline = Pipeline(
    steps=[
        ("preprocess", preprocessor),
        ("model", LGBMClassifier(
            objective="binary",
            random_state=RANDOM_STATE,
            n_jobs=-1
        ))
    ]
)

In [77]:
def lgb_objective(trial):

    params = {
        "model__n_estimators": trial.suggest_int("model__n_estimators", 200, 800),
        "model__learning_rate": trial.suggest_float(
            "model__learning_rate", 0.01, 0.2, log=True
        ),
        "model__max_depth": trial.suggest_int("model__max_depth", 3, 12),
        "model__num_leaves": trial.suggest_int("model__num_leaves", 15, 255),
        "model__subsample": trial.suggest_float("model__subsample", 0.6, 1.0),
        "model__colsample_bytree": trial.suggest_float(
            "model__colsample_bytree", 0.6, 1.0
        ),
        "model__min_child_samples": trial.suggest_int(
            "model__min_child_samples", 10, 50
        )
    }

    model = lgb_pipeline.set_params(**params)

    # handle class imbalance via class_weight
    class_weights = calculate_class_weights(y_train)
    model.set_params(model__class_weight=class_weights)

    model.fit(X_train, y_train)

    y_val_proba = model.predict_proba(X_val)[:, 1]
    return average_precision_score(y_val, y_val_proba)

In [78]:
study_lgb = optuna.create_study(direction="maximize")
study_lgb.optimize(lgb_objective, n_trials=50)

print(study_lgb.best_params)

[I 2026-01-18 20:15:05,072] A new study created in memory with name: no-name-2d1ba957-0e59-4335-a022-955f87726ba3


[LightGBM] [Info] Number of positive: 19415, number of negative: 108363
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.016204 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 603
[LightGBM] [Info] Number of data points in the train set: 127778, number of used features: 31
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Start training from score 0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain

/home/user_stel/miniconda3/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
[I 2026-01-18 20:15:33,852] Trial 0 finished with value: 0.38106566798480235 and parameters: {'model__n_estimators': 671, 'model__learning_rate': 0.041641667365313814, 'model__max_depth': 10, 'model__num_leaves': 199, 'model__subsample': 0.6972240621035143, 'model__colsample_bytree': 0.8160367831315182, 'model__min_child_samples': 11}. Best is trial 0 with value: 0.38106566798480235.


[LightGBM] [Info] Number of positive: 19415, number of negative: 108363
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.011431 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 603
[LightGBM] [Info] Number of data points in the train set: 127778, number of used features: 31
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Start training from score 0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain

/home/user_stel/miniconda3/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
[I 2026-01-18 20:15:39,767] Trial 1 finished with value: 0.3581786410168725 and parameters: {'model__n_estimators': 502, 'model__learning_rate': 0.014717423186071948, 'model__max_depth': 5, 'model__num_leaves': 48, 'model__subsample': 0.9598082644732918, 'model__colsample_bytree': 0.7459497098132004, 'model__min_child_samples': 19}. Best is trial 0 with value: 0.38106566798480235.


...

[LightGBM] [Info] Number of positive: 19415, number of negative: 108363
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002268 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 603
[LightGBM] [Info] Number of data points in the train set: 127778, number of used features: 31
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Start training from score 0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain

/home/user_stel/miniconda3/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
[I 2026-01-18 20:21:26,040] Trial 48 finished with value: 0.3774563296371744 and parameters: {'model__n_estimators': 401, 'model__learning_rate': 0.08945457866679903, 'model__max_depth': 12, 'model__num_leaves': 197, 'model__subsample': 0.9815852894548082, 'model__colsample_bytree': 0.6153101032880338, 'model__min_child_samples': 36}. Best is trial 35 with value: 0.38721251256450995.


[LightGBM] [Info] Number of positive: 19415, number of negative: 108363
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002458 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 603
[LightGBM] [Info] Number of data points in the train set: 127778, number of used features: 31
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Start training from score 0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain

/home/user_stel/miniconda3/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
[I 2026-01-18 20:21:33,508] Trial 49 finished with value: 0.3821840945478919 and parameters: {'model__n_estimators': 347, 'model__learning_rate': 0.022820077888829567, 'model__max_depth': 10, 'model__num_leaves': 143, 'model__subsample': 0.8413059970700424, 'model__colsample_bytree': 0.6713384130746967, 'model__min_child_samples': 26}. Best is trial 35 with value: 0.38721251256450995.


{'model__n_estimators': 411, 'model__learning_rate': 0.04900847530563631, 'model__max_depth': 12, 'model__num_leaves': 152, 'model__subsample': 0.721455577062599, 'model__colsample_bytree': 0.6812661281747796, 'model__min_child_samples': 30}


In [79]:
best_lgb = lgb_pipeline.set_params(**study_lgb.best_params)
print(study_lgb.best_params)

{'model__n_estimators': 411, 'model__learning_rate': 0.04900847530563631, 'model__max_depth': 12, 'model__num_leaves': 152, 'model__subsample': 0.721455577062599, 'model__colsample_bytree': 0.6812661281747796, 'model__min_child_samples': 30}


In [80]:
ml_pipeline.save_model(best_lgb, "lgb_model")

Model saved to: /home/user_stel/project_thesis/thesis_code/data_analysis/models/lgb_model.joblib


PosixPath('/home/user_stel/project_thesis/thesis_code/data_analysis/models/lgb_model.joblib')

# Training on train+val 

In [12]:
# Load all models
models = {
    "rf":  load_model("rf_model"),
    "lr":  load_model("lr_model"),
    "svm": load_model("svm_model"),
    "xgb": load_model("xgb_model"),
    "lgb": load_model("lgb_model")
}

In [13]:
# Prepare combined train+val set
X_trainval = pd.concat([X_train, X_val])
y_trainval = pd.concat([y_train, y_val])

In [14]:
final_results = {}

for name, model in models.items():

    clf = model.named_steps["model"]

    # -----------------------------
    # Handle class imbalance
    # -----------------------------
    if hasattr(clf, "class_weight"):
        class_weights = calculate_class_weights(y_trainval)
        model.set_params(model__class_weight=class_weights)

    elif hasattr(clf, "scale_pos_weight"):
        class_weights = calculate_class_weights(y_trainval)
        scale_pos_weight = class_weights[1] / class_weights[0]
        model.set_params(model__scale_pos_weight=scale_pos_weight)

    # -----------------------------
    # Retrain on train + val
    # -----------------------------
    model.fit(X_trainval, y_trainval)

    # -----------------------------
    # Evaluate on test (ONCE)
    # -----------------------------
    metrics = evaluate_model(
        model,
        X_test,
        y_test,
        split_name=f"{name.upper()} – Final Test"
    )

    final_results[name] = metrics

    # -----------------------------
    # Save final model
    # -----------------------------
    save_model(model, f"{name}_final")


===== RF – Final Test evaluation =====
              precision    recall  f1-score   support

           0      0.922     0.757     0.832     23221
           1      0.330     0.651     0.438      4270

    accuracy                          0.741     27491
   macro avg      0.626     0.704     0.635     27491
weighted avg      0.830     0.741     0.770     27491

Confusion matrix:
[[17585  5636]
 [ 1489  2781]]
Model saved to: /home/user_stel/project_thesis/thesis_code/data_analysis/models/rf_final.joblib

===== LR – Final Test evaluation =====
              precision    recall  f1-score   support

           0      0.927     0.638     0.756     23221
           1      0.270     0.728     0.394      4270

    accuracy                          0.652     27491
   macro avg      0.599     0.683     0.575     27491
weighted avg      0.825     0.652     0.700     27491

Confusion matrix:
[[14816  8405]
 [ 1161  3109]]
Model saved to: /home/user_stel/project_thesis/thesis_code/data_analysis

/home/user_stel/miniconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:305: ConvergenceWarning: Solver terminated early (max_iter=5000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(



===== SVM – Final Test evaluation =====
              precision    recall  f1-score   support

           0      0.845     1.000     0.916     23221
           1      0.000     0.000     0.000      4270

    accuracy                          0.845     27491
   macro avg      0.422     0.500     0.458     27491
weighted avg      0.713     0.845     0.774     27491

Confusion matrix:
[[23221     0]
 [ 4270     0]]
Model saved to: /home/user_stel/project_thesis/thesis_code/data_analysis/models/svm_final.joblib


/home/user_stel/miniconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/user_stel/miniconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/user_stel/miniconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/


===== XGB – Final Test evaluation =====
              precision    recall  f1-score   support

           0      0.933     0.681     0.788     23221
           1      0.298     0.734     0.423      4270

    accuracy                          0.690     27491
   macro avg      0.615     0.708     0.606     27491
weighted avg      0.834     0.690     0.731     27491

Confusion matrix:
[[15822  7399]
 [ 1136  3134]]
Model saved to: /home/user_stel/project_thesis/thesis_code/data_analysis/models/xgb_final.joblib
[LightGBM] [Info] Number of positive: 23613, number of negative: 132786
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.012523 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 603
[LightGBM] [Info] Number of data points in the train set: 156399, number of used features: 31
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Start training from score 0.

/home/user_stel/miniconda3/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/home/user_stel/miniconda3/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(



===== LGB – Final Test evaluation =====
              precision    recall  f1-score   support

           0      0.932     0.688     0.791     23221
           1      0.300     0.726     0.424      4270

    accuracy                          0.694     27491
   macro avg      0.616     0.707     0.608     27491
weighted avg      0.834     0.694     0.734     27491

Confusion matrix:
[[15975  7246]
 [ 1171  3099]]
Model saved to: /home/user_stel/project_thesis/thesis_code/data_analysis/models/lgb_final.joblib


In [16]:
results_df = (
    pd.DataFrame(final_results)
    .T
    .rename_axis("model")
    .reset_index()
)

results_df

,model,average_precision,mcc
0,rf,0.428502,0.321091
1,lr,0.311919,0.268812
2,svm,0.155324,0.000000
3,xgb,0.418933,0.309441
4,lgb,0.414107,0.309321


In [ ]:
from sklearn.metrics import precision_recall_curve
import matplotlib.pyplot as plt

for name, model in models.items():
    y_score = model.predict_proba(X_test)[:, 1]
    precision, recall, _ = precision_recall_curve(y_test, y_score)
    plt.plot(recall, precision, label=name)

plt.xlabel("Recall")
plt.ylabel("Precision")
plt.legend()
plt.title("Precision–Recall curves (test set)")
plt.show()

# Model Eavluation

In [ ]:
models = {
    "RandomForest": best_rf,
    "LogisticRegression": best_lr,
    "SVM": best_svm,
    "XGBoost": best_xgb,
    "LightGBM": best_lgb
}

results = {}

for name, model in models.items():
    results[name] = evaluate_model(
        model,
        X_test,
        y_test,
        split_name=f"{name} – Test"
    )